In [1]:
import pandas as pd
import re,itertools,nltk

In [2]:
tweets = pd.read_csv('Tweets.csv')
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
tweets = tweets[['text','airline_sentiment']]
tweets.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [4]:
# starting with @ followed by one or more word characters[A-z_0-9]
my_parser = re.compile('^@\w+')

In [5]:
##re.findall('^@\w+','@VirginAmerica is good airlines @dheburn')

In [6]:
my_data = map(lambda x : my_parser.findall(x), tweets['text'])
my_complete_data = set(list(itertools.chain.from_iterable(my_data)))
my_complete_data

{'@AlbertBreer',
 '@AmericanAir',
 '@AndrewFallis',
 '@CatfoodBeerGlue',
 '@DeltaAssist',
 '@HillAConlin',
 '@JetBlue',
 '@Jetblue',
 '@NinaDavuluri',
 '@Sb5551',
 '@SouthwestAir',
 '@Southwestair',
 '@USAIRWAYS',
 '@USAirways',
 '@USairways',
 '@United',
 '@UsAirways',
 '@Usairways',
 '@VirginAmerica',
 '@americanair',
 '@djevolutionhd',
 '@eatgregeat',
 '@jetblue',
 '@lindaSWC',
 '@malhoit',
 '@nrhodes85',
 '@ods1819',
 '@sarahpompei',
 '@scm1133',
 '@shannonwoodward',
 '@southwestAir',
 '@southwestair',
 '@timbennettg3',
 '@united',
 '@usairways',
 '@vanessaannz',
 '@virginAmerica',
 '@virginamerica'}

In [7]:
file = open('References.txt','a')
for i in my_complete_data:
    file.write(i+'\n')
file.close()

In [9]:
def extract_np(text):
    my_sentences = nltk.sent_tokenize(text)
    my_sentences = [nltk.word_tokenize(sentence) for sentence in my_sentences]
    my_sentences = [nltk.pos_tag(sentence) for sentence in my_sentences]
    
    # print(my_sentences)
    
    my_grammar = r"NP:{<DT><NN|NNP|NNS|NNPS>*<VB|VBD|VBZ><NN|NNS|NNP|NNPS>}"
    
    my_chunk_parser = nltk.RegexpParser(my_grammar)
    
    my_np_list = [[' '.join(leaf[0] for leaf in tree.leaves()) for tree in my_chunk_parser.parse(sent).subtrees() if tree.label()=='NP'] for sent in my_sentences]
    
    return my_np_list

In [ ]:
for group,sub in tweets.groupby('airline_sentiment'):
    np = map(lambda x: extract_np(x), sub['text'])
    np = list(itertools.chain.from_iterable(np))
    
    all_np = set(list(itertools.chain.from_iterable(np)))
    
    filename = 'NP for '+str(group)+' Review.txt'
    
    file = open(filename, 'a')
    for i in all_np:
        file.write(i+'\n')
    file.close()